In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium
from scipy.stats import normaltest, chi2_contingency, mannwhitneyu, ttest_ind

: 

---
# **Problem that need to be answered in this analysis**


In this capstone project there are some question that needs to be answered:
<br><br>

1. Driver and fleet optimization : How to optimize fleet operations by increasing efficiency and reducing idle time

2. How to identify the busiest times and predict the amount of demand at a given time When and where taxi demand increases

3. Payment Preferences and Driver Income Impact Which payment methods are most popular, and how do they impact driver income

4. Relationship between Trip Fare and Distance Is there a correlation between trip distance and fare amount?

5. Trip Duration and Zone Analysis Which zones (PULocationID, DOLocationID) experience the longest travel times?
6. Impact of Surcharges on Total Fare How do surcharges (MTA taxes, surcharges, tolls) affect total fare?
7. Detecting Fraudulent Trips Are there any unusual trips that could indicate data entry errors or fraud?
8. Comparison between vendor performance Is there a difference between the average trip distance, time and cost of vendor 1 and 2

---
# **The method**

The steps applied in this project will include but are not limited to these highlighted below.

1. Data Preprocessing and Cleaning
- Handle missing values 
- Convert date-time fields into an appropriate format and then extract relevant features from the date: hour, weekday, month.
- Handle outliers in trip_distance and fare_amount-for instance, extremely high values for fare_amount.
- Map location IDs to known NYC boroughs/zones for better insights.
2. Exploratory Data Analysis
- Find peak hours, days of the week, and seasonal trends.
- Map hot zones for pickups and drop-offs based on PULocationID and DOLocationID.
- Look at fare vs. trip distance based on Rate Codes.
- Check different usage patterns among different payment methods.
3. Feature Engineering
New features to be created are as follows:
- trip duration : Time difference between dropoff and pickup.
- Trip Speed: The trip distance divided by the trip time .
- Rush Hour: Provides an indicator flag on whether it is rush hours.

In [ ]:
#Put the data into dataframe format called df_raw
df_raw= pd.read_csv('D:/PURWADHIKA/New York City Taxi and Limousine Trip Record/NYC TLC Trip Record.csv')
display(df_raw.head(),df_raw.tail())

In [8]:
#copy the df raw into df. later on df data will be the base dataframe to modify/cleaned
df = df_raw.copy()

---
**Step 1: Data Understanding:**

In this step, we need to understand the dimension of the data, number of duplicate value, number of null value and the statistic definition of the data for example mean, quantile , etc. 

In [ ]:
print(f'Number of row and column in the dataset df are {df.shape}')
df.info()

In [ ]:
display(df.describe(), df.describe(include='object'))

In [ ]:
df.isna().sum()/df.shape[0]*100

In [ ]:
# find unique data for every column to understand better the data

pd.set_option('display.max_colwidth', 1)

listItem = []
for col in df.columns : listItem.append( [col, df[col].nunique(), df[col].unique()])

tabel1Desc = pd.DataFrame(columns=['Column Name', 'Number of Unique', 'Unique Sample'],data=listItem)
tabel1Desc

In [ ]:
# checking the duplicate of the data
duplicateRows = df[df.duplicated()]
print(duplicateRows)


**Step 1: Data Understanding - Result**

1. Number of row and column in the dataset df are (68211, 20)
2. ehail_fee column has all null value. This column will be dropped because there's no such a pattern that need to be investigated further

3. There are nan value in some of the column with percentage missing data as follow
<br>
    - store_and_fwd_flag 6.339154 %
    - RatecodeID 6.339154 %
    - passenger_count            6.339154  %
    - payment_type               6.339154 %
    - trip_type                  6.353814 %
    - congestion_surcharge       6.339154 %
    <br>
    These data needs to be further investigated so that the treament of this nan value can be decided. The investigation including pattern investigation

4. There are some unappropriate deffinition of data types. Here are 3 data types that  needs to be  adjusted in order 
    - lpep_pickup_datetime and lpep_dropoff_datetime:
        <br>
        Current Type: object (possibly string), Expected Type: datetime
        This data will be converted into  datetime format, because they represent date and time information. This conversion will support other further anlysis such as  calculating trip duration, filtering by specific time ranges, and other time-related analyses.

    - store_and_fwd_flag:
        <br>
        Current Type: object (posibly string with values like "Y" and "N")
        Expected Type: category (or a binary int or bool)
        This data will be converted into a category so that will reduce memory usage, and it allows for efficient filtering.

5.  Found negative value on Improvement_surcharge. Improvement surcharge is basically a fixed addition-almost always $0.30-per-trip charge to help pay for improvements and infrastructure. It's added on top of the fare and does not change based on distance or time of the trip.It is not logicall when it has negative values in this column, it may indicate Data Entry Errors or Data Processing Problems where there could be an error in importing or processing data, especially where some computation or offset would have applied. This data should be handled later.

6. Found negative value on mta_tax: mta_tax is a constant item that is added on each taxi fare in New York City and the money from which is used to benefit Metropolitan Transportation Authority. It should be positive or zero since it is an addition to fare. The negative value probably caused by an error in data entry or processing, such as improper adjustment or transformation of data. The pattern of this data needs to be observed later

7. Found negative value on sxtra: There is no definition of extra on PDF docs. On many cases, The extra field to indicates more surcharges such as rush hour fees or overnight charges. Similar to the mta_tax, these are the costs that come on top of the fare due to particular conditions-peak hours in this case-and would always be positive or zero but not negative. The pattern of this data needs to be observed later

8. There is different category that can be found on RatecodeId. On the data definition, the RatecodeID defined as follow : 
<br> 
    1=Standardrate; 2=JFK; 3=Newark; 4=Nassau or Westchester; 5=Negotiated fare; 6=Group ride
<br><br>
    In reality, however, RatecodeID for the real data contains the following values: [1.0, 5.0, 4.0, 3.0, 2.0, 99.0, nan]. Notably, the value 99.0 seems a bit misplaced while considering the categories defined earlier, as this should be 6 for group rides; in this case, further investigation might be needed.

9. There is no duplicated row on the data

---

---
**Step 2: Missing Value**
<br>
In this step, the pattern of the missing values are observed and the treatment of each missing values is decided

**2.1: ehail_fee**
<br>
Drop the column of ehail_fee and observe the dimension of the data after 

In [ ]:
#code to drop the data
df = df.drop(columns=['ehail_fee'])
#print the information of dataframe after dropping the data
print(df.info())


**2.2 Nan value investigation and treatment**
<br>
These datas has same percentage of missing values.
- store_and_fwd_flag 6.339154 %
- RatecodeID 6.339154 %
- passenger_count            6.339154  %
- payment_type               6.339154 %
- trip_type                  6.353814 %
- congestion_surcharge       6.339154 %


In [ ]:
#plot the data to understand the distribution of the data
plt.figure(figsize=(12,12))
sns.heatmap(df[['VendorID','RatecodeID','PULocationID','DOLocationID','passenger_count','trip_distance','fare_amount','extra','mta_tax','tip_amount','tolls_amount','ehail_fee','improvement_surcharge','total_amount','payment_type','trip_type','congestion_surcharge']].isna())

The plot above showing that the missing value of the data concetrated at lowest row of the data. 
<br>
The other pattern of the missing value will be deeper investigated such as : are these missing value corralated to certain vendor , ratecodeid etc
<br>
To better understand the data the modification of data type lpep_pickup_datetime and lpep_dropoff_datetime are needed so that this data will give better information such as date and times.

In [14]:
#convert the data types to date and time types
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])


In [52]:
#extract features for the dates so that we can understand detail and creates grouping based on the dates
df['pickup_hour'] = pd.to_datetime(df['lpep_pickup_datetime']).dt.hour
df['pickup_day'] = pd.to_datetime(df['lpep_pickup_datetime']).dt.dayofweek
df['pickup_date'] = df['lpep_pickup_datetime'].dt.date
df['pickup_month'] = df['lpep_pickup_datetime'].dt.month      
df['is_weekend'] = df['pickup_day'].apply(lambda x: 1 if x >= 5 else 0)  # Weekend indicator

In [ ]:
df.info()

In [ ]:
missing_data = df.isnull().mean() * 100
print("Percentage of missing values in each column:\n", missing_data)


In [ ]:
# Step 2.1: Analyze patterns in missing data

# Vendor-based missing data analysis
vendor_missing_data = df.groupby('VendorID').apply(lambda x: x.isnull().mean() * 100, include_groups=False)
print("Missing data percentage by VendorID:\n", vendor_missing_data)

In [ ]:
# Plot missing values by VendorID
vendor_missing_data.T.plot(kind='bar', figsize=(10, 6))
plt.ylabel("Percentage of Missing Values")
plt.title("Missing Values by Feature and VendorID")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Calculate missing data percentage by pickup hour
hourly_missing_data = df.groupby('pickup_hour').apply(lambda x: x.isnull().mean() * 100, include_groups=False)
print("Missing data percentage by Pickup Hour:\n", hourly_missing_data)

# Plot missing data by hour
plt.figure(figsize=(10, 6))
sns.heatmap(hourly_missing_data.T, cmap="YlGnBu", annot=True, fmt=".1f")
plt.title("Missing Values by Hour of Day")
plt.xlabel("Hour of Day")
plt.ylabel("Feature")
plt.show()

In [ ]:
# Location-based missing data analysis: check if missing data is location-related
# Calculate missing data percentage by pickup location ID
location_missing_data = df.groupby('PULocationID').apply(lambda x: x.isnull().mean() * 100, include_groups=False)

print("Missing data percentage by Pickup Location ID:\n", location_missing_data)

# Plot missing data by location
plt.figure(figsize=(10, 6))
sns.heatmap(location_missing_data.T, cmap="YlGnBu", annot=False)
plt.title("Missing Values by Pickup Location")
plt.xlabel("Pickup Location ID")
plt.ylabel("Feature")
plt.show()

In [ ]:
# Calculate missing values grouped by 'store_and_fwd_flag'
store_and_fwd_missing_data = df.groupby('store_and_fwd_flag').apply(lambda x: x.isnull().mean() * 100)
store_and_fwd_missing_data = store_and_fwd_missing_data.round(2)

# Display the results
print("Percentage of missing values grouped by store_and_fwd_flag:\n", store_and_fwd_missing_data)


In [ ]:
store_and_fwd_missing_data.T.plot(kind='bar', figsize=(14, 7), colormap='viridis')
plt.title("Percentage of Missing Values by Feature Grouped by 'store_and_fwd_flag'")
plt.ylabel("Percentage of Missing Values")
plt.xlabel("Feature")
plt.xticks(rotation=45)
plt.legend(title="Store and Forward Flag")
plt.show()

In [ ]:
# Step 1.3: Check for other patterns in missing data

# Calculate correlations between missing values to find any relationship patterns
missing_corr = df.isnull().corr()
plt.figure(figsize=(10, 8))
sns.heatmap(missing_corr, annot=True, cmap="coolwarm", center=0)
plt.title("Correlation Between Missing Values of Different Features")
plt.show()

In [ ]:

# Check if shorter or longer trips have more missing values

distance_missing = df.groupby(pd.cut(df['trip_distance'], bins=5), observed=True).apply(lambda x: x.isnull().mean() * 100)

print("Missing data percentage by Trip Distance Bins:\n", distance_missing)

# Plot missing data by trip distance bins
plt.figure(figsize=(10, 6))
sns.heatmap(distance_missing.T, cmap="YlGnBu", annot=True, fmt=".1f")
plt.title("Missing Values by Trip Distance Bins")
plt.xlabel("Trip Distance Bin")
plt.ylabel("Feature")
plt.show()


In [ ]:
# Group by the extracted date and calculate missing values
date_missing_data = df.groupby('pickup_date').apply(lambda x: x.isnull().mean() * 100, include_groups=False)


# Results
print("Percentage of missing values grouped by pickup date:")
print(date_missing_data)

# Plot the data for better understanding
plt.figure(figsize=(14, 7))
sns.heatmap(date_missing_data.T, cmap="YlGnBu", annot=False, cbar_kws={'label': 'Percentage of Missing Values'})
plt.title("Missing Values by Column Grouped by Pickup Date")
plt.xlabel("Pickup Date")
plt.ylabel("Feature")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Filter rows where Passenger_count is 0
zero_passenger_data = df[df['passenger_count'] == 0]

# Calculate the percentage of missing values in the specified columns for zero-passenger rows
zero_passenger_missing = zero_passenger_data[['store_and_fwd_flag', 'RatecodeID', 'passenger_count', 
                                              'payment_type', 'trip_type', 'congestion_surcharge']].isnull().mean() * 100

# Display the results
print("Percentage of missing values for rows with Passenger_count = 0:")
print(zero_passenger_missing)


In [ ]:
# Calculate missing values grouped by RatecodeID for the specified columns
ratecode_missing_data = df.groupby('RatecodeID').apply(
    lambda x: x[['store_and_fwd_flag', 'passenger_count', 'payment_type', 'trip_type', 'congestion_surcharge']].isnull().mean() * 100
)

# Display the results
print("Percentage of missing values grouped by RatecodeID:")
print(ratecode_missing_data)


In [ ]:
# Calculate overall missing values grouped by RatecodeID
overall_missing_data = df.groupby('RatecodeID').apply(lambda x: x.isnull().mean() * 100)

# Display the results
print("Overall percentage of missing values grouped by RatecodeID:")
print(overall_missing_data)


In [ ]:
# Calculate overall missing values grouped by RatecodeID without the DeprecationWarning
overall_missing_data = df.groupby('RatecodeID', group_keys=False).apply(lambda x: x.isnull().mean() * 100)

# Plotting overall missing values by RatecodeID
plt.figure(figsize=(10, 6))
overall_missing_data.plot(kind='bar', color='skyblue')
plt.title("Overall Percentage of Missing Values by RatecodeID")
plt.ylabel("Percentage of Missing Values")
plt.xlabel("RatecodeID")
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()



In [ ]:
# Filter the DataFrame for RatecodeID equal to 99
ratecode_99_data = df[df['RatecodeID'] == 99]
# Calculate the number of missing values for each column
missing_values_count_ratecode_99 = ratecode_99_data.isnull().sum()
#  result
print("Number of missing values for RatecodeID = 99:")
print(missing_values_count_ratecode_99)


After the detail investigation of the missing value grouped by certain variable it can be said that there is no pattern of the missing value.

In [40]:
df_drop = df.dropna(subset=['store_and_fwd_flag','RatecodeID','passenger_count', 'payment_type','trip_type'])

In [ ]:
print(df_drop.info())

In [ ]:
df_drop.isna().sum()/df_drop.shape[0]*100

In [ ]:
display(df_drop.describe(), df_drop.describe(include='object'))

In [ ]:
#melakukan treatment terhadap nilai minus

In [ ]:
# Identify negative values in the specified columns
negative_improvement_surcharge = df[df['improvement_surcharge'] < 0]
negative_mta_tax = df[df['mta_tax'] < 0]
negative_extra = df[df['extra'] < 0]

# Print the counts of negative values
print(f"Negative Improvement Surcharge Count: {negative_improvement_surcharge.shape[0]}")
print(f"Negative MTA Tax Count: {negative_mta_tax.shape[0]}")
print(f"Negative Extra Count: {negative_extra.shape[0]}")

print("Negative Improvement Surcharge Examples:\n", negative_improvement_surcharge.head())
print("Negative MTA Tax Examples:\n", negative_mta_tax.head())
print("Negative Extra Examples:\n", negative_extra.head())


In [ ]:

# melakukan pembuatan boxplot untuk mengetahui outliers, distribution
variables = ['trip_distance', 'fare_amount', 'tip_amount', 'total_amount']

# Set up figure size and plot layout
plt.figure(figsize=(15, 10))

# Loop through variables to create subplots
for i, var in enumerate(variables, 1):
    plt.subplot(2, 2, i)  # Create 2x2 grid of plots
    sns.boxplot(data=df, x=var)
    plt.title(f'Boxplot of {var}')
    plt.ylabel(var)

plt.tight_layout()
plt.show()

In [ ]:
#me;alilam

In [36]:
# melakukan pemisahan pickup
df['pickup_hour'] = df['lpep_pickup_datetime'].dt.hour         # Extract hour
df['pickup_day'] = df['lpep_pickup_datetime'].dt.dayofweek      # 0 = Monday, 6 = Sunday


In [ ]:
print(df[['pickup_hour', 'pickup_day', 'pickup_month', 'is_weekend']].head())

In [ ]:
# Select relevant numerical columns for correlation analysis
numerical_cols = ['fare_amount', 'trip_distance', 'total_amount', 'tip_amount', 'passenger_count', 'congestion_surcharge']

# Calculate correlation matrix
correlation_matrix = df[numerical_cols].corr()

# Plot heatmap to visualize correlations
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix for NYC Taxi Data')
plt.show()

In [ ]:
# Plot pickup hour distribution
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='pickup_hour', palette='Blues')
plt.title('Number of Pickups by Hour')
plt.show()

# Plot pickup day distribution
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='pickup_day', palette='Greens')
plt.title('Number of Pickups by Day of the Week (0 = Monday, 6 = Sunday)')
plt.show()

# Monthly pickup trend
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='pickup_month', palette='Purples')
plt.title('Number of Pickups by Month')
plt.show()

In [ ]:
# Top 10 pickup locations
top_pickups = df['PULocationID'].value_counts().head(10)
top_dropoffs = df['DOLocationID'].value_counts().head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x=top_pickups.index, y=top_pickups.values, palette='Reds')
plt.title('Top 10 Pickup Locations')
plt.xlabel('PULocationID')
plt.ylabel('Number of Pickups')
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(x=top_dropoffs.index, y=top_dropoffs.values, palette='Blues')
plt.title('Top 10 Drop-off Locations')
plt.xlabel('DOLocationID')
plt.ylabel('Number of Drop-offs')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df, x='trip_distance', y='fare_amount', hue='RatecodeID', palette='deep', alpha=0.5)
plt.title('Fare Amount vs. Trip Distance by Rate Code')
plt.xlabel('Trip Distance (miles)')
plt.ylabel('Fare Amount ($)')
plt.legend(title='Rate Code')
plt.show()

In [ ]:


trip_pairs = (
    df.groupby(['PULocationID', 'DOLocationID'])
    .size()
    .reset_index(name='Trip_Count')
    .sort_values(by='Trip_Count', ascending=False)
    .head(10)
)

# Create a new column for route names
trip_pairs['Route'] = trip_pairs['PULocationID'].astype(str) + ' → ' + trip_pairs['DOLocationID'].astype(str)

# Plott
plt.figure(figsize=(12, 6))
sns.barplot(data=trip_pairs, x='Route', y='Trip_Count', palette='viridis')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.title('Top 10 Most Frequent Pickup and Drop-off Pairs')
plt.xlabel('Route (Pickup → Drop-off)')
plt.ylabel('Number of Trips')
plt.show()


In [ ]:
pivot_table = trip_pairs.pivot_table(
    index='PULocationID', 
    columns='DOLocationID', 
    values='Trip_Count', 
    fill_value=0
)

# Plotting the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_table, cmap='YlGnBu', annot=True, fmt='.0f', linewidths=0.5, linecolor='gray')
plt.title('Heatmap of Top 10 Pickup and Drop-off Pairs')
plt.xlabel('Drop-off Location ID')
plt.ylabel('Pickup Location ID')
plt.show()

In [ ]:
# Calculate trip duration in minutes
df['trip_duration'] = (df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']).dt.total_seconds() / 60

# Group by pickup and dropoff zones and calculate average trip duration
longest_trip_zones = (
    df.groupby(['PULocationID', 'DOLocationID'])['trip_duration']
    .mean()
    .reset_index()
    .sort_values(by='trip_duration', ascending=False)
    .head(10)  # Show top 10 longest trip pairs
)

print("Top 10 Pickup and Drop-off Zones with Longest Trip Duration:")
print(longest_trip_zones)

In [ ]:
# Cari jumlah penumpang yang anomalies
unusual_passenger_count = df[(df['passenger_count'] <= 0) | (df['passenger_count'] > 6)]

print("\nPerjalanan dengan jumlah penumpang anomalies:")
print(unusual_passenger_count[['PULocationID', 'DOLocationID', 'passenger_count']])

In [ ]:
# Perjalanan dengan pickup dan drop-off di lokasi yang sama, tapi tarif tinggi
unusual_zone = df[(df['PULocationID'] == df['DOLocationID']) & (df['fare_amount'] > 50)]

print("\nPerjalanan dengan pickup dan drop-off di zona sama tapi tarif tinggi:")
print(unusual_zone[['PULocationID', 'DOLocationID', 'fare_amount', 'trip_duration']])

In [ ]:
# Perjalanan dengan tarif nol atau negatif
unusual_fare = df[df['fare_amount'] <= 0]

print("\nPerjalanan dengan tarif nol atau negatif:")
print(unusual_fare[['PULocationID', 'DOLocationID', 'fare_amount', 'trip_distance']])

# Perjalanan dengan tarif sangat tinggi tetapi jarak pendek
unreasonable_fare = df[(df['fare_amount'] > 200) & (df['trip_distance'] < 5)]

print("\nPerjalanan dengan tarif sangat tinggi dan jarak pendek:")
print(unreasonable_fare[['PULocationID', 'DOLocationID', 'fare_amount', 'trip_distance']])


In [ ]:
#melakukan payment analysis

payment_analysis = (
    df.groupby('payment_type')
    .agg(
        total_income=('total_amount', 'sum'),  # Total income from each payment type
        avg_income_per_trip=('total_amount', 'mean'),  # Average income per trip
        total_tip=('tip_amount', 'sum'),  # Total tips
        avg_tip_per_trip=('tip_amount', 'mean'),  # Average tip per trip
        trip_count=('payment_type', 'size')  # Total number of trips
    )
    .reset_index()
    .sort_values(by='trip_count', ascending=False)  # Sort by trip count
)

# Map payment_type codes to meaningful names
payment_type_mapping = {
    1: 'Credit Card',
    2: 'Cash',
    3: 'No Charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided Trip'
}
payment_analysis['payment_type'] = payment_analysis['payment_type'].map(payment_type_mapping)

print("Payment Method Analysis:")
print(payment_analysis)

In [ ]:
#Melakukan kalulasi trip dalam menit
df['trip_duration'] = (df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']).dt.total_seconds() / 60

# Group by VendorID 
vendor_performance = (
    df.groupby('VendorID')
    .agg(
        avg_trip_duration=('trip_duration', 'mean'),
        avg_trip_distance=('trip_distance', 'mean'),
        total_trips=('VendorID', 'size'),
        avg_fare_amount=('fare_amount', 'mean'),
        avg_tip_amount=('tip_amount', 'mean')
    )
    .reset_index()
)

print("Vendor Performance Comparison:")
print(vendor_performance)

--
**Gunakan melb_data.csv (Melbourne House Data) dengan:**
Do the rides that originate from certain areas tend to generate more revenue?
Does the distance of a trip considerably impact the received tip?
Does the average fare differ with modes of payment, such as cash versus credit card?
1. Does Time of Day Impact Fare Amounts?
Hypothesis: The objective is to test if there is a significant difference between the amount of fares at different parts of the day-for example, morning and evening.
Example:
H₀: The mean amount of fare is the same for different times of day.
H₁: There is a significant difference in the average fare amount by time of day, such as rush hours versus off-peak hours.
This type of analysis could be helpful in determining the best times to make more drivers available in order to increase revenues.

2. Is There Seasonal Variation in Trip Demand?
Hypothesis: Test for a significant difference in the number of trips across seasons or months.
Example:
H₀: The number of trips is equal across all seasons.
H₁: There is some significant difference in the number of trips across seasons.
If you find seasonality, that could be helpful to adjust fleet size or marketing effort in high demand periods.

3. Is the Number of Passengers Influencing the Fare Amount?
Hypothesis: Check if larger groups are willing to pay more, or if the number of passengers influences the fare amount pricing.
Example:
H₀: Number of passengers and fare amount are independent.
H₁: There is a significant difference in fare amount depending on the number of passengers.
This could be enlightening about shared vs. individual ride fare strategies.
 
4. Does the Tip Amount Differ Due to Payment Type?
Hypothesis: Use a t-test to determine if credit card vs. cash determines different tip amounts.
Example:
H₀: The average fare tip is equal for credit card payments as well as cash payments.
H₁: Credit card and cash are yielding significantly different average tip amounts.
Knowing these patterns in tipping due to payment type may provide an indication of how to incentivize certain types of payments.

5. Is Trip Distance a More Important Predictor of Total Amounts Compared with Other Variables?
Hypothesis: Determine if trip distance is most influential with respect to total trip revenue compared with other variables such as duration of trip or rate code.
Example:
Ho: Trip distance is no more influential determinant of total amount compared to other factors.
H1: Trip distance is a more influential determinant of the total amount compared to other factors.
The test described will support or reject the supposition that longer trips will always generate more money and therefore shape the pricing and distance strategies.

6. Are Fares from Airport Locations Significantly Higher Than Non-Airport Locations?
Hypothesis: Determine if the average fares for trips that begin or end in major airport locations, such as JFK or Newark, are higher compared to others. Example: H₀: Average fares in airport and non-airport locations are equal. H₁: Average fares for airport locations are much higher. This will be useful insight that feeds targeted services or dynamic pricing models for airport trips.

7. Do Store-and-Forward Trips (Disconnected Trips) Differ in Revenue?
Hypothesis-Check whether the trips which were flagged as store-and-forward due to temporary network disconnects, generate different revenues compared to regular trips. Example: H0 : the average fare amount of the store-and-forward trips is equal to the fare amount in regular trips. H1: there is a significant difference between store-and-forward trips and regular trips in terms of fare amount. This might show if the network connectivity is affecting the value of fares, that could have an implication in the collection process.


Test 1: Difference in Average Fare by Payment Type
This test will check if payment type impacts the average fare amount.

H₀: The average fare is the same for all payment types.
H₁: There is a significant difference in average fare between at least two payment types.
Code Example for ANOVA Test:
python
Copy code

In [ ]:
from scipy import stats

# Filter dataset for valid payment types
data_filtered = df[df['Payment_type'].isin([1, 2])]  

# Extract fare amounts by payment type
fare_credit = data_filtered[data_filtered['Payment_type'] == 1]['Fare_amount']
fare_cash = data_filtered[data_filtered['Payment_type'] == 2]['Fare_amount']

# Perform ANOVA
f_stat, p_value = stats.f_oneway(fare_credit, fare_cash)
print("ANOVA F-statistic:", f_stat)
print("ANOVA p-value:", p_value)

# Interpret the p-value
if p_value < 0.05:
    print("Reject the null hypothesis: Significant difference in average fare by payment type.")
else:
    print("Fail to reject the null hypothesis: No significant difference in average fare by payment type.")

Test 2: Impact of Trip Distance on Tip Amount

In [ ]:
# Use Pearson correlation test
correlation, p_value = stats.pearsonr(df['Trip_distance'], df['Tip_amount'])
print("Correlation coefficient:", correlation)
print("p-value:", p_value)

# Interpret the p-value
if p_value < 0.05:
    print("Reject the null hypothesis: Significant correlation between trip distance and tip amount.")
else:
    print("Fail to reject the null hypothesis: No significant correlation between trip distance and tip amount.")


Test 3: Do High-Volume Zones Generate Higher Total Revenue?
This test will check if there’s a significant difference in total revenue for trips originating in high-demand zones versus low-demand zones.

In [ ]:
# Define high-demand zones (top 10% by trip count)
top_zones = df['PULocationID'].value_counts().head(int(len(data) * 0.1)).index
df['is_high_demand'] = df['PULocationID'].apply(lambda x: 1 if x in top_zones else 0)

# Separate data into high-demand and low-demand zone trips
high_demand_fares = df[df['is_high_demand'] == 1]['Total_amount']
low_demand_fares = df[df['is_high_demand'] == 0]['Total_amount']

# Perform t-test
t_stat, p_value = stats.ttest_ind(high_demand_fares, low_demand_fares, equal_var=False)
print("t-statistic:", t_stat)
print("p-value:", p_value)

# Interpret the p-value
if p_value < 0.05:
    print("Reject the null hypothesis: High-demand zones have significantly different total revenue.")
else:
    print("Fail to reject the null hypothesis: No significant difference in total revenue between high and low-demand zones.")

3. Statistical Inference and Confidence Intervals
For added robustness, calculate confidence intervals for mean fare or revenue in high-demand vs. low-demand zones.


In [ ]:
# Calculate 95% confidence interval for high-demand zones
high_demand_mean = high_demand_fares.mean()
high_demand_std = high_demand_fares.std()
high_demand_n = len(high_demand_fares)
high_demand_conf_interval = stats.norm.interval(0.95, loc=high_demand_mean, scale=high_demand_std / np.sqrt(high_demand_n))

print("95% Confidence Interval for High-Demand Zone Revenue:", high_demand_conf_interval)